In [ ]:
run ../data_utils.py

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import constants as C
import matplotlib.pyplot as plt

In [ ]:
def tables_from_category(category):
    with open('../../data/pickle_files/%s.pickle' % category, 'rb') as f:
        return pickle.load(f)

qa_table, reviews_table = tables_from_category(C.VIDEO_GAMES)

#Question with maximum length

In [ ]:
mask = qa_table['questionText'].str.len() == 20
result = qa_table.loc[mask]['questionText']
question = result.iloc[1]

In [ ]:
print question

In [ ]:
stopset_u = set(stopwords.words('english'))
stopset = set([str(sw) for sw in stopset_u])

tokens = nltk.word_tokenize(question)
tokens = [w.lower() for w in tokens if not w.lower() in stopset]

print len(tokens)

#Questions containing "Read More"

In [ ]:
mask = qa_table['answerText'].str.contains("Read More")
result = qa_table.loc[mask]['questionText']

#Preprocess Data

In [ ]:
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
qa_data = getDF('../../data/answers_multiple/QA_Video_Games.json.gz')
reviews_df = getDF('../../data/reviews_small/reviews_Video_Games_5.json.gz')

In [ ]:
def questionsToJson(questions_list):
    new_questions_list = []
    for question in questions_list:
        
        new_question = {}
        new_question[C.TEXT] = question[C.QUESTION_TEXT]
        new_question[C.TIME] = question[C.QUESTION_TIME]
        new_question[C.TYPE] = question[C.QUESTION_TYPE]
        
        new_answers = []
        for answer in question[ANSWERS]:
            
            new_answer = {}
            new_answer[C.TEXT] = answer[C.ANSWER_TEXT]
            new_answer[C.TIME] = answer[C.ANSWER_TIME]
            
            scores = str(answer[C.HELPFUL])[1:-1].split(',')
            new_answer[C.HELPFUL] = int(scores[0].strip())
            new_answer[C.UNHELPFUL] = int(scores[1].strip())
        new_answers.append(new_answer)
        
        new_question[C.ANSWERS] = new_answers
        new_questions_list.append(new_question)
    
    return new_questions_list

In [ ]:
qa_data[C.QUESTIONS_LIST] = qa_data[QUESTIONS].apply(questionsToJson)
qa_data = qa_data[[C.ASIN, C.QUESTIONS_LIST]]

In [ ]:
def filterReviewText(str):
    return str

In [ ]:
def convertRowToReviewJson(row):
        json = {}
        json['text'] = filterReviewText(row['reviewText'])
        
        scores = str(row['helpful'])[1:-1].split(',')
        json['helpful'] = int(scores[0].strip())
        json['unhelpful'] = int(scores[1].strip())
        
        json['time'] = row['reviewTime']
        return json

In [ ]:
reviews_df['reviews'] = reviews_df[['reviewText', 'helpful', 'reviewTime']].apply(convertRowToReviewJson, axis=1)

In [ ]:
def convertRowsToList(columns):
        return columns['reviews'].tolist()

In [ ]:
reviews_filtered_df = reviews_df.groupby('asin').apply(convertRowsToList).reset_index()
reviews_filtered_df.columns = ['asin', 'reviews']

In [ ]:
qa_reviews = pd.merge(qa_data, reviews_filtered_df, on=[C.ASIN, C.ASIN])[['asin', 'questionJson']]

In [ ]:
qa_reviews.head()

In [ ]:
C.JSON_DATA_PATH = '../../data/json_data'

def raw_dataframe(category):
    with open('%s/%s.pickle' % (C.JSON_DATA_PATH, category), 'rb') as f:
        return pd.read_pickle(f)

In [ ]:
df = raw_dataframe(C.VIDEO_GAMES)

In [ ]:
df['questionsList'][0]

In [ ]:
def getLengths(row):
    reviewsList = row['reviewsList']
    reviewsLengthList = []
    for review in reviewsList:
        tokens = review['text'].split()
        reviewsLengthList.append(len(tokens))
    
    questionsList = row['questionsList']
    questionsLengthList = []
    answersLengthList = []
    
    for question in questionsList:
        tokens = question['text'].split()
        questionsLengthList.append(len(tokens))
        
        answersList = question['answers']
        for answer in answersList:
            tokens = answer['text'].split()
            answersLengthList.append(len(tokens))
    
    return (reviewsLengthList, questionsLengthList, answersLengthList)

In [ ]:
df['lengthsList'] = df[['reviewsList', 'questionsList']].apply(getLengths, axis=1)

In [ ]:
reviewsLengthList = []
questionsLengthList = []
answersLengthList = []

for (r, q, a) in df['lengthsList']:
    reviewsLengthList.extend(r)
    questionsLengthList.extend(q)
    answersLengthList.extend(a)

In [ ]:
len(answersLengthList)

In [ ]:
plt.hist(questionsLengthList, bins=20)

In [ ]:
plt.hist(answersLengthList, bins=20)

In [ ]:
plt.show()